# Соединение ключевых слов для Проектного практикума - Учебная задача (семестр 3)

Черновое решение

## Установка зависимостей (если не установлены ранее)

In [1]:
# Установка зависимостей для параллельной обработки и блокнота
# %pip install pandas ipywidgets jupyter IProgress protobuf joblib tdqm

## Импорты, константы и настройки

In [2]:
# Импорт зависимостей
from os.path import join as path_join
from os import walk as dir_walk

from re import split as re_split

from tqdm import tqdm


import pandas as pd

In [3]:
# Константы
PATH_TO_KW_PARTS: str = '.././data/cleared/key_words/'
NAME_OF_KW_PARTS: str = 'key_words'

PATH_TO_DS: str = '.././data/cleared/'
DS_FILENAME: str = 'cleared_dataset.csv'

## Чтение датасета

In [4]:
# Чтение основного датасета
df_data: pd.DataFrame = pd.read_csv(
    path_join(PATH_TO_DS, DS_FILENAME),
    sep=';',
    index_col='indx')
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499800 entries, 0 to 499999
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   rating_int    499800 non-null  int64 
 1   rubrics_list  499800 non-null  object
 2   cleared_text  499800 non-null  object
dtypes: int64(1), object(2)
memory usage: 15.3+ MB


In [5]:
# Добавление пустой колонки для ключевых слов
df_data['key_words'] = pd.Series(dtype='object')
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499800 entries, 0 to 499999
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   rating_int    499800 non-null  int64 
 1   rubrics_list  499800 non-null  object
 2   cleared_text  499800 non-null  object
 3   key_words     0 non-null       object
dtypes: int64(1), object(3)
memory usage: 19.1+ MB


## Чтение ключевых слов

In [6]:
# Чтение списка файлов с ключевыми словами и их именных индексов
kw_files: list[tuple[str, int, int]] = []
for (dirpath, dirnames, filenames) in dir_walk(PATH_TO_KW_PARTS):
    for filename in tqdm(filenames, 'Поиск файлов: '):
        if not filename.startswith(NAME_OF_KW_PARTS): continue
        # Определение именных индексов
        name_parts: list[str] = re_split(r'[_\.]', filename)
        kw_file: tuple[str, int, int] = (filename, int(name_parts[2]), int(name_parts[3]))
        kw_files.append(kw_file)

# Сортировка по индексам
kw_files.sort(key=lambda f: f[1])

print(f'Прочитано {len(kw_files)} файла(ов)')

Поиск файлов: 100%|██████████| 1000/1000 [00:00<?, ?it/s]

Прочитано 1000 файла(ов)


In [7]:
# Вывод диапазонов, которые не прочитаны
missing_files: list[tuple[int, int]] = []
tmp_kw_file: tuple[int, int] = None
for filename, f, t in kw_files:
    if tmp_kw_file is None:
        tmp_kw_file = (f, t)
        continue
    if f == tmp_kw_file[1]:
        tmp_kw_file = (f, t)
        continue
    missing_files.append((tmp_kw_file[1], f))
    tmp_kw_file = (f, t)
        

print(missing_files)

[]


In [8]:
def parse_kw_file(filename: str) -> list[tuple[int, list[str]]]:
    '''Функция чтения ключевых слов из файла'''
    df_kw: pd.DataFrame = None
    # try:
    df_kw = pd.read_csv(
        path_join(PATH_TO_KW_PARTS, filename),
        sep=';',
        index_col='indx')
    result_list: list[tuple[int, list[str]]] = []
    for indx, kw in df_kw['0'].items():
        result: tuple[int, list[str]] = (indx, kw)
        result_list.append(result)
    return result_list

In [9]:
# Чтение данных из файла, анализ индексов внутри
result_list: list[tuple[int, list[str]]] = []
for kw_file, fr, to in tqdm(kw_files, 'Чтение файлов: '):
    kw_content: list[tuple[int, list[str]]] = parse_kw_file(kw_file)
    for indx, kw in kw_content:
        if not indx in df_data.index:
            print(indx, kw)
            continue
        df_data.loc[indx, 'key_words'] = kw

Чтение файлов: 100%|██████████| 1000/1000 [00:34<00:00, 28.68it/s]


In [10]:
# Список индексов с пустыми ключевыми словами
empty_kw_indexes: list[int] = df_data.loc[df_data['key_words'].isnull()].index.to_list()
print(f'Не прочитано {len(empty_kw_indexes)} индексов')

Не прочитано 0 индексов


## Сохранение датасета

In [11]:
# Сохранение датасета 
df_data.to_csv(
    path_join(PATH_TO_DS, f'kw_{DS_FILENAME}'),
    sep=';', 
    encoding='utf-8-sig',
    index_label='indx')

In [34]:
# Сохранение ключевых слов
kw_unic_list: list[str] = []
for kw in tqdm(df_data['key_words'].to_list()):
    kws = list(eval(kw))
    if len(kws) == 0: 
        continue
    kw_unic_list.extend(kws)

# Формирование набора
kw_unic_series: pd.Series = pd.Series(
    sorted(kw_unic_list)) \
        .value_counts()
kw_unic_series

100%|██████████| 499800/499800 [00:03<00:00, 137772.53it/s]


интернет-маркетинг       50755
удалённая работа         49618
гаджеты                  47657
дизайн                   42155
интернет-магазин         38861
                         ...  
ёлткарино                    1
5 калифорнийка               1
ёлагин                       1
ёлка большая пушистая        1
ёлка для гиков               1
Name: count, Length: 70387, dtype: int64

In [35]:
# Сохранение набора ключевых слов
kw_unic_series.to_csv(
    path_join(PATH_TO_DS, 'kws.csv'),
    sep=';', 
    encoding='utf-8-sig',
    index_label='indx')